Feature Extraction:
    1. Create Feature vector with all words as 1 and 0. (Good for naive bayes classifier)
    2. Term Freq Vector.(Contains freq of words in vocal). (Good for SVM)
    3. 
    

In [1]:
import pandas as pd
import nltk
from nltk import FreqDist
from heapq import nlargest

In [2]:
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from string import punctuation

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
def acc(a,b):
    print('Accuracy(test Data): ',accuracy_score(a,b))
    print('Confusion Matrix: \n',pd.DataFrame(confusion_matrix(a,b)))
    print('Classification Repost:\n',classification_report(a,b,digits=3))


In [3]:
df = pd.read_csv("data2.csv",engine='python')
tweets = list(df.tweet)
_stopwords = set(stopwords.words('english') + list(punctuation) +
                 ["million","billion","year","millions","billions",
                  "y/y","'s'","''","�",'``',1,2,3,4,5,6,7,8,9,0,'a','b','c','d','e','f','ƒ'])

In [72]:
df.columns

Index(['id', 'label', 'tweet', 'new_twt'], dtype='object')

In [112]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df[['id','tweet']],df['label'], test_size=0.20, random_state=0)

In [113]:
def add_words_col(X_train,col,new_col):
    tweets = list(X_train[col])
    texts = []
    for tweet in tweets:
        words = word_tokenize(tweet.lower())
        words=[word for word in words if word not in _stopwords]
        texts.append(words)
    X_train[new_col] = texts

In [114]:
add_words_col(X_train,'tweet','new_twt')

In [116]:
len(freq)

11231

In [117]:
#Creating Vocab: fro X_train:
x = []
for text in X_train['new_twt']:
    x = x+text
freq = FreqDist(x)
vocab = nlargest(8000,freq,key=freq.get)

In [118]:
def extract_features(tweet):
    t_words = set(tweet)
    features = {}
    for word in vocab:
        features[word] = (word in t_words)
    return features

In [119]:
#Getting training data tuples.
train_data = []
for row,label in zip(X_train['new_twt'],y_train):
    train_data.append((row,label))

In [120]:
def getTrainedNaiveBayesClassifier(extract_features,train_data):
    train_features = nltk.classify.apply_features(extract_features,train_data)
    trainedNBClassifier = nltk.NaiveBayesClassifier.train(train_features)
    return trainedNBClassifier

In [121]:
trainedNBClassifier = getTrainedNaiveBayesClassifier(extract_features,train_data)

In [122]:
def NBSentimentCalculator(tweet):
    tweet = tweet.split()
    feature = extract_features(tweet)
    return trainedNBClassifier.classify(feature)

In [123]:
y_pred = []
for tweet in X_test['tweet']:
    y_pred.append(NBSentimentCalculator(tweet))

In [124]:
NBSentimentCalculator('Great movie. I loved it.')

0

In [125]:
acc(y_test,y_pred)

Accuracy(test Data):  0.8007626310772163
Confusion Matrix: 
      0    1
0  560   39
1  170  280
Classification Repost:
               precision    recall  f1-score   support

           0      0.767     0.935     0.843       599
           1      0.878     0.622     0.728       450

    accuracy                          0.801      1049
   macro avg      0.822     0.779     0.785      1049
weighted avg      0.815     0.801     0.794      1049

